<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/DF_real_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from google.colab import drive

# 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def gaussian_blur(img, level):
    # 256px 크기에 맞게 시각적 블러 강도 보정
    sigmas = [0.3, 0.6, 0.9, 1.2, 1.5] # 기존 [1, 2, 3, 4, 5]에서 하향
    sigma = sigmas[level-1]
    return cv2.GaussianBlur(img, (0, 0), sigma)

def gaussian_noise(img, level):
    # 노이즈 입자가 얼굴 이목구비를 가리지 않도록 강도 조정
    sigmas = [2, 5, 8, 12, 18] # 기존 [5, 10, 15, 20, 25]에서 하향
    sigma = sigmas[level-1]
    noise = np.random.normal(0, sigma, img.shape)
    out = img.astype(np.float32) + noise
    return np.clip(out, 0, 255).astype(np.uint8)

def jpeg_compression(img, level):
    # 작은 이미지에서 Quality 10은 너무 파괴적임. 시각적 유사도에 맞춰 상향
    qualities = [80, 65, 50, 35, 20] # 기존 [30, 20, 10, 5, 2]에서 상향
    quality = qualities[level-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 1)

def block_wise(img, level):
    """지적하신 '회색 픽셀/국소 블록' 양상을 재현하는 함수"""
    out = img.copy()
    h, w = img.shape[:2]
    # 레벨에 따라 블록의 개수와 크기를 조절 (256px 기준 최적화)
    block_counts = [3, 8, 15, 25, 40]
    block_sizes = [4, 8, 12, 16, 20]

    count = block_counts[level-1]
    size = block_sizes[level-1]

    for _ in range(count):
        y, x = np.random.randint(0, h - size), np.random.randint(0, w - size)
        # f000_1.jpg처럼 회색 계열의 매크로블록 에러 시뮬레이션
        if np.random.rand() > 0.5:
            out[y:y+size, x:x+size] = 128 # 회색 블록
        else:
            # 주변부를 뭉개는 블록
            roi = out[y:y+size, x:x+size]
            out[y:y+size, x:x+size] = cv2.blur(roi, (size, size))
    return out

def color_contrast(img, level):
    # 색상은 해상도 영향을 안 받지만, 레벨별 편차를 더 세밀하게 조정
    factors = [0.95, 0.85, 0.75, 0.65, 0.55]
    return np.clip(img.astype(np.float32) * factors[level-1], 0, 255).astype(np.uint8)

def color_saturation(img, level):
    factors = [0.95, 0.85, 0.75, 0.65, 0.55]
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:, :, 1] *= factors[level-1]
    return cv2.cvtColor(np.clip(hsv, 0, 255).astype(np.uint8), cv2.COLOR_HSV2BGR)

def video_compression(img, level):
    # VC는 JPEG보다 약간 더 뭉개지는 느낌으로 설정
    qualities = [70, 55, 40, 25, 10]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), qualities[level-1]]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 1)

def apply_perturbation(img, dist_type, level):
    if dist_type == 'GB': return gaussian_blur(img, level)
    elif dist_type == 'GNC': return gaussian_noise(img, level)
    elif dist_type == 'JPEG': return jpeg_compression(img, level)
    elif dist_type == 'BW': return block_wise(img, level)
    elif dist_type == 'CC': return color_contrast(img, level)
    elif dist_type == 'CS': return color_saturation(img, level)
    elif dist_type == 'VC': return video_compression(img, level)
    return img

In [3]:
# 압축풀기
!unzip -q /content/drive/MyDrive/HECTO/Dataset/images_data.zip -d /content/data_local

In [4]:
# --- 경로 설정 ---
# 1. 원본 FF++ Real 사진 경로
FF_REAL_BASE = '/content/data_local/content/drive/MyDrive/HECTO/Dataset/Images/FF_frames/Real'
# 2. 메타 파일(.txt)이 들어있는 경로
META_ROOT = '/content/drive/MyDrive/HECTO/Dataset/DF_lists'
# 3. 결과물을 저장할 DF Real 경로
SAVE_BASE = '/content/drive/MyDrive/HECTO/Dataset/DF_Frames_Final/Real'

# --- 사용자 요청 설정 (ID 범위 및 레벨) ---
configs = [
    ('manipulated_videos_end_to_end_level_3_meta.txt', range(0, 500), 'level_3'),
    ('manipulated_videos_end_to_end_level_4_meta.txt', range(500, 1000), 'level_4'),
    ('manipulated_videos_end_to_end_level_5_meta.txt', range(0, 500), 'level_5'),
    ('manipulated_videos_end_to_end_mix_2_distortions_meta.txt', range(500, 1000), 'mix_2')
]

def load_dist_map(meta_path, id_range):
    dist_map = {}
    if not os.path.exists(meta_path):
        print(f"파일을 찾을 수 없음: {meta_path}")
        return dist_map
    with open(meta_path, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            if len(parts) < 2: continue
            v_id_str = os.path.basename(parts[0]).split('_')[0]
            try:
                if int(v_id_str) in id_range:
                    d_info = parts[1].split(':')
                    dist_map[v_id_str] = (d_info[0], int(d_info[1]))
            except: continue
    return dist_map

In [5]:
# --- 실행 루프 ---
for meta_file, id_range, dist_folder_name in configs:
    print(f"\n🚀 [작업 시작] {dist_folder_name} (ID 범위: {id_range.start}~{id_range.stop-1})")

    meta_path = os.path.join(META_ROOT, meta_file)
    dist_map = load_dist_map(meta_path, id_range)

    if not dist_map:
        print(f"⚠️ {meta_file}에서 매칭되는 ID가 없습니다. 건너뜁니다.")
        continue

    for v_id, (d_type, d_level) in tqdm(dist_map.items(), desc=f"Processing {dist_folder_name}"):
        src_folder = os.path.join(FF_REAL_BASE, v_id)
        if not os.path.exists(src_folder):
            continue

        # 저장 경로 구조: .../Real/level_3/000_std_3/
        dst_folder = os.path.join(SAVE_BASE, dist_folder_name, f"{v_id}_std_{d_level}")

        # 이어하기: 이미 15장 이상 있으면 스킵
        if os.path.exists(dst_folder) and len(os.listdir(dst_folder)) >= 15:
            continue
        os.makedirs(dst_folder, exist_ok=True)

        # 원본에서 최대 15프레임 가져오기
        frames = sorted([f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])[:15]

        for img_name in frames:
            img_path = os.path.join(src_folder, img_name)
            img = cv2.imread(img_path)
            if img is None: continue

            # 왜곡 적용
            distorted_img = apply_perturbation(img, d_type, d_level)

            # 최종 저장
            cv2.imwrite(os.path.join(dst_folder, img_name), distorted_img)

print("\n✨ 모든 Real 데이터 왜곡 전처리가 완료되었습니다!")


🚀 [작업 시작] level_3 (ID 범위: 0~499)


Processing level_3: 100%|██████████| 500/500 [02:00<00:00,  4.14it/s]



🚀 [작업 시작] level_4 (ID 범위: 500~999)


Processing level_4: 100%|██████████| 500/500 [02:04<00:00,  4.01it/s]



🚀 [작업 시작] level_5 (ID 범위: 0~499)


Processing level_5: 100%|██████████| 500/500 [02:02<00:00,  4.07it/s]



🚀 [작업 시작] mix_2 (ID 범위: 500~999)


Processing mix_2: 100%|██████████| 500/500 [02:10<00:00,  3.82it/s]


✨ 모든 Real 데이터 왜곡 전처리가 완료되었습니다!
